# Get exemplary pages for schools exemplary of each topic

- Author: Jaren Haber
- Institution: UC Berkeley; Georgetown University
- Date created: January 2021
- Date last edited: January 2021

Description: Use page counts to rank school pages and look at those with highest scores for a given topic. Start with lists of distinctive terms and distinctive school websites for each topic. Getting distinctive pages is essential for text extraction and stimulus generation for our follow-up survey experiment.

## Initialize

In [3]:
#!pip install nltk
#import nltk; nltk.download('stopwords'); nltk.download('punkt')

In [190]:
# Import packages
import pandas as pd # For working with DataFrames
import gc # To accelerate loading pickle files
import os, datetime, re, sys
from tqdm import tqdm
tqdm.pandas()

import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer # approximate but effective (and common) method of stemming words
ps = PorterStemmer()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [9]:
# Load functions from data mgmt/tools directory:
cwd = os.getcwd()
root = str.replace(cwd, "text_analysis/filter_top_pages", "")
sys.path.insert(0, root + "data_management/tools")

# For displaying basic DF info, storing DFs for memory efficiency, and loading a filtered DF:
from df_tools import check_df, convert_df, load_filtered_df, replace_df_nulls

# For quickly loading & saving pickle files in Python:
from quickpickle import quickpickle_dump, quickpickle_load 

# For saving and loading text lists to/from file:
from textlist_file import write_list, load_list 

In [214]:
# Set file paths
thisday = datetime.date.today().strftime("%m%d%y")

charters_path = root + "misc_data/charters_2015.pkl"
ex_urls_path = root + "text_analysis/topic_modeling/stm_example_urls_011220.csv"
topwords_paths = [fp for fp in os.listdir(root + "text_analysis/topic_modeling/") if fp.startswith("top_words")]

topwords_output_path = root + "text_analysis/topic_modeling/stm_top_pages_{}.tsv".format(str(thisday))

## Define text helpers

In [16]:
# Create list of punctuation
import string # for one method of eliminating punctuation
punctlist = list(string.punctuation) # assign list of common punctuation symbols
punctlist+=['*','•','©','–','–','``','’','“','”','...','»',"''",'..._...','--','×','|_','_','§','…','⎫'] # Add a few more punctuations also common in web text
punctlist.remove('-') ; punctlist.remove("'")
punctstr = "".join([char for char in list(set(punctlist))])
punctstr = re.sub(r'_--', '', punctstr)

# Create unicode list
unicode_list  = []
for i in range(1000,3000):
    unicode_list.append(chr(i))

# Create stopword list
stopword_list = list(set(stopwords.words('english')))

In [203]:
def clean_sentence(messy_string, 
                   unicodelist = unicode_list, 
                   punctuations = punctstr, 
                   stopwords = stopword_list): 
    """Removes numbers, stopwords, emails, URLs, unicode characters, hex characters, and punctuation from a sentence 
    separated by whitespaces. Returns a tokenized, stemmed, cleaned list of words from the sentence.
    
    Args: 
        messy_string (str): may include spaces and punctuation
        unicodelist (list of str): list of unicode symbols
        punctuations (str): string containing punctuation marks
        stopwords (list of str): list of stopwords
    Returns: 
        Cleaned & tokenized sentence, i.e. a list of cleaned, lower-case, one-word strings"""
    
    
    # remove newline characters and "|" characters
    sentence = messy_string.replace("|", " ").replace("\n", " ")
    
    #replace \\x, \\u, \\b, or - followed by any character or anything that ends with \u2605
    #then replace \\x, \\t and then get rid of whitespace
    sentence = re.sub(r"\\x.*|\\u.*|\\b.*|-|\u2605$", "", messy_string.replace(u"\xa0", u" ").replace(u"\\t", u" ").strip(" "))
    
    #get rid of hex character like \xa0\ adn a\x80
    sentence = re.sub(r'[^\x00-\x7f]',r'', sentence) #replace anything that starts with a hex character 
    
    #code that basically removes all elements that appear in the unicode_list (looks like r'u1000|u10001|')
    sentence = re.sub(r'|'.join(map(re.escape, unicodelist)), '', sentence) #removes unicode
    li_text = []
    
    for word in re.split('\s', sentence): #split sentence by space, lower case
        word = word.lower()
        if ((word not in stopwords) and (not word.isdigit()) and ("@" not in word)) : #gets rid of the numbers and emails
        
            #gets rid of urls 
            if ((not word.startswith(('http', 'https', 'www'))) and (not word.endswith(('.com', '.net', '.gov', '.org')))):
                if ((not word.startswith('//')) and not word.endswith(('.jpg', '.pdf', 'png', 'jpeg', 'php'))): 
                                        
                    word = re.sub(r'['+punctuations+']|am|pm', r'', word) #get rid of punctuation, and the words am and pm
                    word = ps.stem(word) #stem word
                    li_text.append(word)
                    
    sentence = ' '.join(li_text) #joins all words together
    sentence = sentence.strip() #remove beginning and ending white space in string
    return sentence

In [161]:
# Read in distinctive words for each selected topic. 
# We have 4 metrics for distinctiveness: score, prob, frex, and lift
topwords_dfs = [pd.read_csv(
    root + "text_analysis/topic_modeling/" + fp, low_memory = False) 
                for fp in topwords_paths]

# Merge these DFs by row
topwords_df = pd.concat([
    df.rename(columns = {"Unnamed: 0":"TOPIC"}).set_index(keys = "TOPIC") 
    for df in topwords_dfs], axis=1)

topwords_df

,score.1,score.2,score.3,score.4,score.5,score.6,score.7,score.8,score.9,score.10,...,prob.41,prob.42,prob.43,prob.44,prob.45,prob.46,prob.47,prob.48,prob.49,prob.50
TOPIC,,,,,,,,,,,,,,,,,,,,,
1,cours,credit,colleg,student,high,onlin,gilbert,creek,graduat,transcript,...,admiss,scienc,transcript,contact,opportun,includ,avail,semest,advanc,elect
2,athlet,club,galleri,basketbal,ace,denver,varsiti,staff,parent,girl,...,powerschool,soccer,coach,ace,advisori,volleybal,meet,denver,transpar,director
3,elementari,school,middl,river,high,bay,haven,north,academi,oak,...,view,learn,technolog,servic,program,dragon,primari,career,magnet,height
4,like,get,said,love,realli,work,help,know,want,kid,...,ask,thank,alway,friend,join,give,next,pleas,import,stori
5,campus,imagin,click,enrol,suppli,newslett,school,parent,pto,form,...,bus,faculti,mission,welcom,vision,import,fax,board,payment,question
6,discrimin,sex,disabl,complaint,color,constel,school,marit,race,sexual,...,pleas,usda,reserv,state,civil,sexual,orient,child,request,receiv
7,art,scienc,student,project,learn,music,stem,club,math,engin,...,present,product,game,robot,find,well,build,field,time,health
8,spambot,javascript,window,madison,html,file,pane,score,gradebook,enabl,...,size,chang,must,set,indic,term,descript,list,edit,download
9,kipp,edlio,email,sorri,los,verifi,send,messag,angel,authent,...,street,portal,staff,search,famili,parent,island,atlanta,subscrib,posit


## Load data

In [162]:
# Load data, keeping only relevant columns
df = load_filtered_df(
    charters_path, 
    ["WEBTEXT", "CMO_WEBTEXT", "URL", "SCH_NAME", "INQUIRY_COUNT", "INQUIRY_RATIO", 
     "NUMWORDS", "NUMPAGES", "NCESSCH", "SY_STATUS15"])

# rows and cols:  (10965, 10)
# duplicates by NCESSCH: 0

Columns and # missing cases (if any): 
URL: 3828 missing
SCH_NAME: 8421 missing
INQUIRY_COUNT
INQUIRY_RATIO: 4103 missing
NUMWORDS
NUMPAGES
NCESSCH
SY_STATUS15: 3619 missing
WEBTEXT
CMO_WEBTEXT


In [163]:
# Sanity check: Detect duplicates
print(len(df["NCESSCH"])) # Number of values in NCESSCH column
print(len(df["NCESSCH"])-len(df.drop_duplicates(subset="NCESSCH"))) # Method 1 to detect number of duplicates
print(len(list(df["NCESSCH"]))-len(list(set(df["NCESSCH"])))) # Method 2 to detect number of duplicates

10965
0
0


In [164]:
# Load file with exemplary schools for each key topic in STM
ex_urls_df = pd.read_csv(
    ex_urls_path, low_memory = False, header = 0, 
    usecols = ['TOPIC', 'NCESSCH', 'URL']).sort_values(by = 'TOPIC')
ex_urls_df

,NCESSCH,TOPIC,URL
81,80336001839,1,http://www.swecollege.org/
27,61152013697,1,https://emsofl.com/
34,61962006745,1,http://ktlcharterschool.com/
43,62380011518,1,http://www.sscs.cc/
59,63474011464,1,http://ogcs.org/
...,...,...,...
83,80336006517,28,http://uprepschool.org/
85,80441006638,28,http://www.salidadelsolacademy.org/
88,110003500253,28,http://ccpcs.org/
38,62271010851,28,http://www.gertzresslerhigh.org/


In [165]:
# Merge exemplary schools with their WEBTEXT from web-crawled charter school data
df = pd.merge(df[["WEBTEXT", "NCESSCH"]], ex_urls_df, how = "right", on = "NCESSCH")
df = df[['WEBTEXT', 'TOPIC', 'NCESSCH', 'URL']]
df

,WEBTEXT,TOPIC,NCESSCH,URL
0,"[(http://www.swecollege.org/, False, 0, We are...",1,8.033600e+10,http://www.swecollege.org/
1,"[(https://emsofl.com/college-prep/, False, 1, ...",1,6.115201e+10,https://emsofl.com/
2,"[(http://ktlcharterschool.com/, False, 0, Cont...",1,6.196201e+10,http://ktlcharterschool.com/
3,"[(http://www.sscs.cc/, False, 0, In The News.....",1,6.238001e+10,http://www.sscs.cc/
4,"[(http://ogcs.org/, False, 0, In The News...\n...",1,6.347401e+10,http://ogcs.org/
...,...,...,...,...
275,[(http://uprepschool.org/campuses/arapahoe-str...,28,8.033601e+10,http://uprepschool.org/
276,"[(http://www.salidadelsolacademy.org/, False, ...",28,8.044101e+10,http://www.salidadelsolacademy.org/
277,"[(https://www.ccpcs.org/, False, 0, \r\tJavaSc...",28,1.100035e+11,http://ccpcs.org/
278,"[(https://www.gertzresslerhigh.org/, False, 0,...",28,6.227101e+10,http://www.gertzresslerhigh.org/


## Score pages by keywords

In [166]:
def score_page(txt, 
               keywords):
    '''
    Assign a score to a page based on the normalized count of keywords it contains.
    
    Args:
        txt (str): text of a web page
        terms (list of str): list of terms to look for in pages (e.g., distinctive words for a given topic)
    Returns:
        score (float, range 0-1): how much this page reflects these keywords
    '''
    
    txt = clean_sentence(txt) # clean sentence
    num_words = len(txt.split()) # split on whitespace to find the number of words
    
    total_count = 0
    for word in keywords: 
        count = len(re.findall(str(word) + str("\W"), txt))
        total_count += count
        
    # normalize by page length and take the log to avoid very small values
    if num_words == 0:
        score = 0
    else:
        score = np.log(total_count / num_words)
    
    return score

In [198]:
def get_top_pages(pagelist, 
                  keywords, 
                  n = 3):
    '''
    Get the n most representative pages in a list of website pages (pagelist) by counting frequency of key terms.
    
    Args:
        pagelist (list of list of str): list of pages, each page a quadruple: (URL, is_pdf, depth, text_str)
        n (int): number of pages to select
        keywords (list of str): list of keywords to look for in pages (e.g., distinctive words for a given topic)
    Returns:
        pagelist_selected (list of list of str): n most representative pages given topwords, each page a quadruple (as with input)
    '''
    
    scores_dict = {}
    
    for i, page in enumerate(pagelist):
        page_text = page[3]
        score = score_page(page_text, keywords)
        scores_dict[page] = score
        
    # If number of pages < n, sort/order what pages there are and return the lot
    if len(pagelist) <= n:
        pagelist_selected = sorted(scores_dict, key=scores_dict.get, reverse=True)[:len(pagelist)] # return all pages (sorted)
    else:
        pagelist_selected = sorted(scores_dict, key=scores_dict.get, reverse=True)[:n] # return n top pages
        
    return pagelist_selected

In [195]:
# Test out page scoring/ranking function
keywords = topwords_df.iloc[3,1:].tolist()
get_top_pages(df["WEBTEXT"].iloc[0], keywords = keywords, n = 3)

<ipython-input-166-fe6ee105459d>:25: RuntimeWarning: divide by zero encountered in log
  score = np.log(total_count / num_words)


[('http://www.swecollege.org/whatis.html',
  'False',
  '1',
  'arly college high school is a bold approach, based on the principle that academic rigor, combined with the opportunity to save time and money, is a powerful motivator for students to work hard and meet serious intellectual challenges. Early college high schools blend high school and college in a rigorous yet supportive program, compressing the time it takes to complete a high school diploma and the first two years of colleg'),
 ('http://www.swecollege.org/contactus.html',
  'False',
  '1',
  '\t\n3001 S. Federal Blvd. Denver, CO 80236  \nPhone: 303-935-5473\nFAX: 303-935-5591\t  \n                    Southwest Early College                '),
 ('http://www.swecollege.org/announcements.html',
  'False',
  '1',
  ' DPS SchoolChoice Expo Week – DPS is holding five regional school expos rather than one large districtwide expo, as we have in the past. These expos will take place the week of Jan. 22, 2018. More information \nher

In [133]:
# For reference (I don't use this function)
def df_top_page_getter(row, 
                       n = 5):
    '''
    Applies get_top_pages() function to input DataFrame and returns an array (new column) with selected pages.
    
    Args:
        row (DataFrame): must have 'WEBTEXT' and 'TOPIC' columns
        n (int): number of pages to select
    Returns:
        TOP_PAGES: new column with top pages for each row/site
    '''
    
    print(row[3])
    #print(row[0][1:])
    print("PAUSE")
    TOP_PAGES = get_top_pages(
        row[0], 
        keywords = row[1][1:],
        n = n)
    
    return TOP_PAGES


# Apply function to get top pages for each distinctive school per topic
#df['TOP_PAGES'] = df.apply(lambda row: df_top_page_getter(row, n = 5))

In [199]:
# For each topic, get distinctive terms, then use these to get top pages.
# Add these top pages (one set per topic) to list of DFs.
# At the end, concatenate all these top-pages-DFs, then merge this with main DF.

numpages = 5 # Number of pages to rank and save
top_pages_dfs = [] # initialize list of DFs

for topicnum in set(df["TOPIC"].tolist()):
    topicterms = set(topwords_df.iloc[topicnum-1, :].tolist()) # get topic terms; the (- 1) adjusts for zero-indexing in iloc
    
    df_topic = df[df["TOPIC"] == topicnum] # extract exemplary schools for this topic
    df_topic["PAGE_COUNT"] = df_topic["WEBTEXT"].progress_apply( # count pages for this school
        lambda pagelist: len(pagelist))
    
    # Call get_top_pages function on exemplary schools for this topic
    tqdm.pandas(desc = "Getting top pages for T{}".format(str(topicnum)))
    df_topic["TOP_PAGES"] = df_topic["WEBTEXT"].progress_apply(
        lambda pagelist: get_top_pages(
            pagelist, 
            keywords = topicterms, 
            n = numpages))
    
    top_pages_dfs.append(df_topic)
    
top_pages_df = pd.concat(top_pages_dfs)

Getting top pages for T28: 100%|██████████| 20/20 [00:00<00:00, 17292.53it/s]
<ipython-input-199-af82d14b5741>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_topic["PAGE_COUNT"] = df_topic["WEBTEXT"].progress_apply( # count pages for this school
/opt/conda/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Getting top pages for T1: 100%|██████████| 20/20 [00:11<00:00,  1.70it/s]
<ipython-input-199-af82d14b5741>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Getting top pages for T13: 100%|██████████| 20/20 [00:00<00:00, 27804.47it/s]
<ipython-input-199-af82d14b5741>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_topic["PAGE_COUNT"] = df_topic["WEBTEXT"].progress_apply( # count pages for this school
/opt/conda/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Getting top pages for T17: 100%|██████████| 20/20 [00:23<00:00,  1.20s/it]
<ipython-input-199-af82d14b5741>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

## Finalize data object and save

In [201]:
# Drop huge WEBTEXT column
top_pages_df = top_pages_df.drop(columns = "WEBTEXT")
top_pages_df

,TOPIC,NCESSCH,URL,PAGE_COUNT,TOP_PAGES
0,1,8.033600e+10,http://www.swecollege.org/,14,"[(http://www.swecollege.org/whatis.html, False..."
1,1,6.115201e+10,https://emsofl.com/,61,"[(https://emsofl.com/program/a-g-courses/, Fal..."
2,1,6.196201e+10,http://ktlcharterschool.com/,98,[(http://ktlcharterschool.com/2016/03/15/concu...
3,1,6.238001e+10,http://www.sscs.cc/,94,[(http://www.sscs.cc/index.php/ss-educationalr...
4,1,6.347401e+10,http://ogcs.org/,91,[(http://ogcs.org/index.php/og-educationalreso...
...,...,...,...,...,...
275,28,8.033601e+10,http://uprepschool.org/,13,"[(http://uprepschool.org/es/join-our-team/, Fa..."
276,28,8.044101e+10,http://www.salidadelsolacademy.org/,26,[(http://www.salidadelsolacademy.org/%20https:...
277,28,1.100035e+11,http://ccpcs.org/,311,[(https://www.ccpcs.org/application/files/7415...
278,28,6.227101e+10,http://www.gertzresslerhigh.org/,297,[(https://www.gertzresslerhigh.org/apps/events...


In [210]:
# Get original columns back, will merge in new ones
ex_urls_df_all = pd.read_csv(
    ex_urls_path, low_memory = False, header = 0)
ex_urls_df_all = ex_urls_df_all.drop(columns = ["Unnamed: 0", "TOPIC", "URL"])
list(ex_urls_df_all)

['NCESSCH', 'INDEX', 'SHARED_URL', 'NAME1516', 'ADDRESS1516']

In [221]:
# Merge in new columns: PAGE_COUNT and TOP_PAGES
ex_urls_df_all = pd.merge(ex_urls_df_all, 
                          top_pages_df, 
                          how = "outer", 
                          on = "NCESSCH")

# Sort by topic then page count
ex_urls_df_all = ex_urls_df_all.sort_values(
    by = ['TOPIC', 'PAGE_COUNT'], 
    axis = 0)
ex_urls_df_all

,NCESSCH,INDEX,SHARED_URL,NAME1516,ADDRESS1516,TOPIC,URL,PAGE_COUNT,TOP_PAGES
14,40063603011,253,0,Blueprint High School,"670 North Arizona Avenue , Chandler AZ 85225.0",1,http://www.blueprinthighschool.org/,1,"[(http://www.blueprinthighschool.org/, False, ..."
105,160000501056,2290,0,RICHARD MCKENNA CHARTER HIGH SCHOOL ALTERNATIVE,"1993 E 8TH N STE 105 SUITE 105 , MOUNTAIN HOM...",1,https://www.rmckenna.org/online.html,1,"[(https://www.rmckenna.org/online.html, False,..."
162,350013700819,3291,0,SOUTHWEST SECONDARY LEARNING CENTER,"10301 CANDELARIA NE , ALBUQUERQUE NM 87112.0",1,https://www.sslc-nm.com,6,[(http://www.sslc-nm.com/schools/southwest_sec...
64,64158012111,1311,0,West Sacramento Early College Prep Charter,"1504 Fallbrook St. , West Sacramento CA 95691.0",1,http://www.westsacprep.org/,10,"[(http://www.westsacprep.org/, False, 0, Engag..."
159,330327100679,3163,0,Virtual Learning Academy (H),"30 Linden St. , Exeter NH 3833.0",1,https://vlacs.org/middle-high-school/,11,"[(https://vlacs.org/adult-education/, False, 0..."
...,...,...,...,...,...,...,...,...,...
101,130123003687,2227,0,Unidos Dual Language School,"4475 Hendrix Dr , Forest Park GA 30297.0",28,http://198.clayton.k12.ga.us/,137,[(https://198.clayton.k12.ga.us/news_noticias/...
127,220015902331,2475,0,Lycee Francais de la Nouvelle-Orleans,"5951 Patton Street , New Orleans LA 70115.0",28,http://www.lfno.org/,266,"[(http://www.lfno.org/, False, 0, ), (http://w..."
38,62271010851,816,0,Alliance Gertz-Ressler Richard Merkin 6-12 Com...,"2023 S. Union Ave. , Los Angeles CA 90007.0",28,http://www.gertzresslerhigh.org/,297,[(https://www.gertzresslerhigh.org/apps/events...
88,110003500253,1726,1,Capital City PCS Lower,"100 Peabody St NW , Washington DC 20011.0",28,http://ccpcs.org/,311,[(https://www.ccpcs.org/application/files/7415...


In [222]:
# Save resulting data
ex_urls_df_all.to_csv(topwords_output_path, sep="\t")